In [1]:
import sqlite3
import pandas as pd

# Tạo database SQLite trong bộ nhớ
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()



In [3]:
#Bài 1
cursor.execute("DROP TABLE IF EXISTS score;")

cursor.execute("""
CREATE TABLE score (
    Day TEXT,
    A INTEGER,
    B INTEGER,
    C INTEGER
)
""")

data = [
    ('Day 1', 1, 9, 3),
    ('Day 2', 5, 6, 7),
    ('Day 3', 3, 7, 5),
    ('Day 4', 4, 5, 6),
]

cursor.executemany("INSERT INTO score VALUES (?, ?, ?, ?)", data)
conn.commit()
query = """
SELECT
    (COUNT(*) * SUM(A * B) - SUM(A) * SUM(B)) * 1.0 /
    (SQRT(COUNT(*) * SUM(A*A) - SUM(A)*SUM(A)) *
     SQRT(COUNT(*) * SUM(B*B) - SUM(B)*SUM(B))) AS r_ab
FROM score
"""
print(pd.read_sql_query(query, conn))


       r_ab
0 -0.885714


In [4]:
import pandas as pd
from scipy.stats import chi2_contingency

# Dữ liệu điểm số
data = {
    'A': [8, 7.5, 8, 6],
    'B': [9, 8.5, 7, 7],
    'C': [7, 7, 8, 5]
}
df = pd.DataFrame(data, index=['Day 1', 'Day 2', 'Day 3', 'Day 4'])

# Thực hiện kiểm định Chi bình phương
chi2, p, dof, expected = chi2_contingency(df)

print("Chi-squared:", chi2)
print("p-value:", p)
print("Degrees of freedom:", dof)
print("Expected frequencies:\n", expected)

# Kết luận
if p < 0.05:
    print("Có sự khác biệt đáng kể giữa các mẫu.")
else:
    print("Không có sự khác biệt đáng kể giữa các mẫu.")


Chi-squared: 0.4719322848365669
p-value: 0.9981634697516962
Degrees of freedom: 6
Expected frequencies:
 [[8.04545455 8.59090909 7.36363636]
 [7.71022727 8.23295455 7.05681818]
 [7.71022727 8.23295455 7.05681818]
 [6.03409091 6.44318182 5.52272727]]
Không có sự khác biệt đáng kể giữa các mẫu.


In [5]:

# Tạo bảng flights với departure_time là số nguyên (VD: 830 = 08:30)
cursor.execute('''
CREATE TABLE flights (
    flight_id INTEGER PRIMARY KEY,
    departure_time INTEGER
)
''')

# Dữ liệu mẫu
flight_times = [
    (1, 830),
    (2, 1445),
    (3, 50),
    (4, 123),
    (5, 2359),
    (6, 0)
]

# Chèn dữ liệu vào bảng
cursor.executemany('INSERT INTO flights (flight_id, departure_time) VALUES (?, ?)', flight_times)

# Truy vấn và chuyển đổi sang định dạng HH:MM
query = '''
SELECT
    flight_id,
    departure_time,
    printf('%02d:%02d', departure_time / 100, departure_time % 100) AS formatted_time
FROM flights
ORDER BY flight_id;
'''

# Thực thi và hiển thị kết quả
df = pd.read_sql_query(query, conn)
print(df)

# Đóng kết nối
conn.close()


   flight_id  departure_time formatted_time
0          1             830          08:30
1          2            1445          14:45
2          3              50          00:50
3          4             123          01:23
4          5            2359          23:59
5          6               0          00:00


In [10]:
#bzi 4
import numpy as np
import pandas as pd

# Dữ liệu đầu vào
data = [10, 11, 12, 13, 11, 9, 15, 8]

# Tính median
median = np.median(data)

# Tính độ lệch tuyệt đối
abs_deviation = [abs(x - median) for x in data]

# Tính MAD
mad = np.median(abs_deviation)

# Ngưỡng phát hiện ngoại lệ
threshold = 1.5 * mad

# Đánh dấu ngoại lệ
outliers = [abs(x - median) > threshold for x in data]

# Tổng hợp kết quả
df = pd.DataFrame({
    "value": data,
    "abs_dev": abs_deviation,
    "is_outlier": outliers
})

# Thêm median, MAD, threshold vào bảng cho rõ ràng
df["median"] = median
df["MAD"] = mad
df["threshold"] = threshold

print(df)


   value  abs_dev  is_outlier  median  MAD  threshold
0     10      1.0       False    11.0  1.5       2.25
1     11      0.0       False    11.0  1.5       2.25
2     12      1.0       False    11.0  1.5       2.25
3     13      2.0       False    11.0  1.5       2.25
4     11      0.0       False    11.0  1.5       2.25
5      9      2.0       False    11.0  1.5       2.25
6     15      4.0        True    11.0  1.5       2.25
7      8      3.0        True    11.0  1.5       2.25


In [11]:
import sqlite3
import pandas as pd

# Kết nối SQLite
conn = sqlite3.connect(":memory:")
cursor = conn.cursor()

# Tạo bảng Patient
cursor.execute("""
CREATE TABLE Patient (
    id INTEGER PRIMARY KEY,
    last_name TEXT,
    weight INTEGER,
    height INTEGER
)
""")

# Thêm dữ liệu mẫu
patients = [
    ('Nguyen', 65, 170),
    ('Nguyen', 65, 168),
    ('Tran', 70, 170),
    ('Le', 65, 165),
    ('Nguyen', 60, 170)
]
cursor.executemany("INSERT INTO Patient (last_name, weight, height) VALUES (?, ?, ?)", patients)

# Truy vấn so sánh Boolean
query = """
SELECT
    p1.id AS id1, p1.last_name AS name1, p1.weight AS weight1,
    p2.id AS id2, p2.last_name AS name2, p2.weight AS weight2,
    CASE WHEN p1.last_name = p2.last_name THEN 1 ELSE 0 END AS match_last_name,
    CASE WHEN p1.weight = p2.weight THEN 1 ELSE 0 END AS match_weight,
    (CASE WHEN p1.last_name = p2.last_name THEN 1 ELSE 0 END +
     CASE WHEN p1.weight = p2.weight THEN 1 ELSE 0 END) AS match_score
FROM Patient p1
JOIN Patient p2 ON p1.id < p2.id
ORDER BY match_score DESC
"""

# Đọc kết quả
df = pd.read_sql_query(query, conn)
print(df)

conn.close()


   id1   name1  weight1  id2   name2  weight2  match_last_name  match_weight  \
0    1  Nguyen       65    2  Nguyen       65                1             1   
1    1  Nguyen       65    4      Le       65                0             1   
2    1  Nguyen       65    5  Nguyen       60                1             0   
3    2  Nguyen       65    4      Le       65                0             1   
4    2  Nguyen       65    5  Nguyen       60                1             0   
5    1  Nguyen       65    3    Tran       70                0             0   
6    2  Nguyen       65    3    Tran       70                0             0   
7    3    Tran       70    4      Le       65                0             0   
8    3    Tran       70    5  Nguyen       60                0             0   
9    4      Le       65    5  Nguyen       60                0             0   

   match_score  
0            2  
1            1  
2            1  
3            1  
4            1  
5            0  
